## 一般的mnist和一般的cnn(print parameters)

In [1]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)



model = Net()

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')



criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


8,490 total parameters.
8,490 training parameters.
1:	Test set: Average loss: 0.0033, Accuracy: 9387/10000 (94%)
2:	Test set: Average loss: 0.0020, Accuracy: 9611/10000 (96%)
3:	Test set: Average loss: 0.0017, Accuracy: 9680/10000 (97%)
4:	Test set: Average loss: 0.0015, Accuracy: 9709/10000 (97%)
5:	Test set: Average loss: 0.0012, Accuracy: 9748/10000 (97%)
6:	Test set: Average loss: 0.0012, Accuracy: 9762/10000 (98%)
7:	Test set: Average loss: 0.0011, Accuracy: 9786/10000 (98%)
8:	Test set: Average loss: 0.0010, Accuracy: 9789/10000 (98%)
9:	Test set: Average loss: 0.0011, Accuracy: 9781/10000 (98%)
10:	Test set: Average loss: 0.0009, Accuracy: 9800/10000 (98%)
11:	Test set: Average loss: 0.0010, Accuracy: 9808/10000 (98%)
12:	Test set: Average loss: 0.0009, Accuracy: 9827/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9825/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9816/10000 (98%)
15:	Test set: Average loss: 0.0009, Accuracy: 9824/10000 (98%)
16:	Test set:

## GPU Enabled

In [3]:
%%time
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Check if cuda available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=0)


model = Net()
model = model.to(device)
criterion = criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        # sum up batch loss
        #test_loss += F.nll_loss(output, target, reduction='sum').item()
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(str(epoch)+":\t"+'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 21):
    train(epoch)
    test(epoch)
print('done')


1:	Test set: Average loss: 0.0034, Accuracy: 9364/10000 (94%)
2:	Test set: Average loss: 0.0022, Accuracy: 9589/10000 (96%)
3:	Test set: Average loss: 0.0018, Accuracy: 9654/10000 (97%)
4:	Test set: Average loss: 0.0015, Accuracy: 9711/10000 (97%)
5:	Test set: Average loss: 0.0013, Accuracy: 9740/10000 (97%)
6:	Test set: Average loss: 0.0013, Accuracy: 9741/10000 (97%)
7:	Test set: Average loss: 0.0011, Accuracy: 9776/10000 (98%)
8:	Test set: Average loss: 0.0011, Accuracy: 9772/10000 (98%)
9:	Test set: Average loss: 0.0011, Accuracy: 9775/10000 (98%)
10:	Test set: Average loss: 0.0010, Accuracy: 9797/10000 (98%)
11:	Test set: Average loss: 0.0010, Accuracy: 9789/10000 (98%)
12:	Test set: Average loss: 0.0010, Accuracy: 9798/10000 (98%)
13:	Test set: Average loss: 0.0009, Accuracy: 9809/10000 (98%)
14:	Test set: Average loss: 0.0009, Accuracy: 9808/10000 (98%)
15:	Test set: Average loss: 0.0009, Accuracy: 9810/10000 (98%)
16:	Test set: Average loss: 0.0009, Accuracy: 9820/10000 (98%)
1